In [1]:
import pandas as pd
import requests 
import os
import math
from xml.etree.ElementTree import fromstring, ElementTree
from datetime import datetime
import json
import statistics 
from collections import Counter 

In [2]:
TrafficEvents = None

chunk_size = 1000
max_distance_from_node = 50 # Meter
number_of_nearest_node = 5 # count

OSM_BASIC_URL = "https://api.openstreetmap.org/api/0.6/"
BASE_PATH = "/home/amin/CETI/RoadConstruction/TrafficEventData/part/"
    
index_folder = 0
index_start_point_file = 0
index_end_point_file = 411


In [3]:
def read_data(path):
    data = None
    print("read csv data is started with file {}".format(path))
    data = pd.read_csv(path)
    print("reading is finished length is {} ".format(len(data)))
    return data

def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0]

In [8]:
class OSM(object):
    @staticmethod
    def get_node_details_by_node_id(node_id):
        URL = OSM_BASIC_URL + "node/{}".format(node_id)
        response = requests.get(url = URL)
        result_tree = ElementTree.fromstring(response.content)
        return result_tree, response.content
    
    @staticmethod
    def get_all_ways_contain_node_by_node_id(node_id):
        URL = OSM_BASIC_URL + "node/{}/ways".format(node_id)
        # print(URL)
        response = requests.get(url = URL)
        # result_tree = ElementTree.fromstring(response.content)
        result_tree = ElementTree(fromstring(response.content))

        return result_tree, response.content
    
    @staticmethod
    def get_type_of_way_contain_node_by_node_id(node_id):
        way_type = "None" 
        if node_id == 0:
            return way_type
        try:
            result_tree, content = OSM.get_all_ways_contain_node_by_node_id(node_id)
            root = result_tree.getroot()
            for child in root:
                if child.tag == "way":
                    for child1 in child:
                        if child1.tag == "tag" and child1.attrib["k"] == "highway":
                            way_type = child1.attrib["v"]
                            return way_type
        except:
            print("ERROR: node id {} has error".format(node_id))
            
        return way_type

In [ ]:
out_dir = BASE_PATH + "_" + str(index_folder) + "_" 
if not os.path.exists(out_dir):
     os.mkdir(out_dir)

for i in range(index_start_point_file, index_end_point_file):
    type_of_ways_all_points = []
    print("file id is {}".format(i))
    print("start time is {}".format(datetime.now()))
    path = BASE_PATH + "_" + str(index_folder) + "/" + str(i) + "_file_"+ str(i) + ".csv"
    
    traffic_events_data = read_data(path)
    for index, data in traffic_events_data.iterrows():
        start_points_node_id = json.loads(data['nearst_nodes_ids_of_start_point'])
        end_points_node_id = json.loads(data['nearst_nodes_ids_of_end_point'])
        
        start_points_node_id = start_points_node_id[:3]
        end_points_node_id = end_points_node_id[:3]
        
        ways_id = set(start_points_node_id + end_points_node_id)
        type_of_ways = []
        while len(ways_id) > 0:
            node_id = ways_id.pop()
            type_of_way = OSM.get_type_of_way_contain_node_by_node_id(node_id)
            type_of_ways.append(type_of_way)
        type_of_road = most_frequent(type_of_ways)
        if index % 50 == 0:
            print("number: {} is {}".format(index, type_of_road))
            print("Time is {}".format(datetime.now()))
        type_of_ways_all_points.append(type_of_road)
    
    traffic_events_data["type_of_roads"] = type_of_ways_all_points
    
    output_file_url = BASE_PATH + "_" + str(index_folder) + "_/" + str(i) + "_file_"+ str(i) + ".csv"
    traffic_events_data.to_csv (output_file_url, index = False, header=True)
    print(traffic_events_data.columns)
    print("{} done".format(output_file_url))
    print("end time is {}".format(datetime.now()))

file id is 0
start time is 2020-08-25 23:37:32.655652
read csv data is started with file /home/amin/CETI/RoadConstruction/TrafficEventData/part/_0/0_file_0.csv
reading is finished length is 500 
number: 0 is motorway
Time is 2020-08-25 23:37:36.017818
number: 50 is secondary
Time is 2020-08-25 23:40:18.012110
number: 100 is motorway
Time is 2020-08-25 23:42:52.808108
number: 150 is motorway
Time is 2020-08-25 23:45:26.873704
number: 200 is motorway
Time is 2020-08-25 23:47:56.714361
number: 250 is trunk
Time is 2020-08-25 23:50:25.416271
number: 300 is motorway
Time is 2020-08-25 23:52:59.381624
number: 350 is motorway
Time is 2020-08-25 23:55:36.260369
number: 400 is primary
Time is 2020-08-25 23:58:05.276325
number: 450 is secondary
Time is 2020-08-26 00:00:34.634728
Index(['Id', 'Source', 'Type', 'EventCode', 'Severity', 'StartPoint_Lat',
       'StartPoint_Lng', 'EndPoint_Lat', 'EndPoint_Lng', 'Distance(mi)',
       'StartTime', 'EndTime', 'ImpactingRoad', 'DelayFromTypical(mins)',

In [ ]:
aa = read_data(output_file_url)

In [ ]:
aa.head()